### How many keyed histograms have identical keys across processes?

In [bug 1380880](https://bugzilla.mozilla.org/show_bug.cgi?id=1380880) :billm found that keyed histograms recorded on different processes would be aggregated together if their keys matched.

How often does this happen in practice? How long has this been happening?

In [ ]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py

from plotly.graph_objs import *
from moztelemetry import get_pings_properties, get_one_ping_per_client
from moztelemetry.dataset import Dataset

%matplotlib inline

### Which keyed histograms share keys across processes?

The whole child-process client aggregation thing was introduced by [bug 1218576](https://bugzilla.mozilla.org/show_bug.cgi?id=1218576) back in September of 2016 for Firefox 52. So that's the earliest this could have started.

In [ ]:
pings = Dataset.from_source("telemetry") \
    .where(docType='main') \
    .where(appVersion=lambda x: x.startswith("52")) \
    .where(appUpdateChannel="nightly") \
    .records(sc, sample=0.1)

In [ ]:
def set_of_hgram_key_tuples(payload):
    return set((kh_name, key) for (kh_name, v) in payload['keyedHistograms'].items() for key in v.keys())

def get_problem_combos(aping):
    parent_tuples = set_of_hgram_key_tuples(aping['payload'])
    child_tuples = [set_of_hgram_key_tuples(pp) for (process_name, pp) in aping['payload'].get('processes', {}).items() if 'keyedHistograms' in pp]
    problem_combos = set.intersection(*(child_tuples + [parent_tuples])) if len(child_tuples) else set()
    return problem_combos

In [ ]:
problem_combos = pings.flatMap(get_problem_combos)

In [ ]:
problem_combos.cache()

Alright, let's get a list of the most commonly-seen histograms:

In [ ]:
sorted(problem_combos.map(lambda c: (c[0], 1)).countByKey().iteritems(), key=lambda x: x[1], reverse=True)

More verbosely, what are the 20 most-commonly-seen histogram,key pairs:

In [ ]:
sorted(problem_combos.map(lambda c: (c, 1)).countByKey().iteritems(), key=lambda x: x[1], reverse=True)[:20]

#### Has this been a problem this whole time?

From earlier we note that `IPC_MESSAGE_SIZE/PLayerTransaction::Msg_Update` is the most common "present on multiple processes" combination.

To see if we've had this problem the whole time, how many pings have these messages in both parent and content, and whose histograms have identical sums?

In [ ]:
def relevant_ping(p):
    parent = p.get('payload', {}).get('keyedHistograms', {}).get('IPC_MESSAGE_SIZE', {}).get('PLayerTransaction::Msg_Update')
    content = p.get('payload', {}).get('processes', {}).get('content', {}).get('keyedHistograms', {}).get('IPC_MESSAGE_SIZE', {}).get('PLayerTransaction::Msg_Update')
    return parent is not None and content is not None and parent['sum'] == content['sum']
    
relevant_pings = pings.filter(relevant_ping)

In [ ]:
relevant_pings.count()

Yup, it appears as though we've had this problem since nightly/52.

### How about recently?

In [ ]:
modern_pings = Dataset.from_source("telemetry") \
    .where(docType='main') \
    .where(submissionDate="20170716") \
    .records(sc, sample=0.01)

In [ ]:
modern_combos = modern_pings.flatMap(get_problem_combos)

In [ ]:
modern_combos.cache()

In [ ]:
sorted(modern_combos.map(lambda c: (c[0], 1)).countByKey().iteritems(), key=lambda x: x[1], reverse=True)

In [ ]:
sorted(modern_combos.map(lambda c: (c, 1)).countByKey().iteritems(), key=lambda x: x[1], reverse=True)[:20]

The behaviour still exists, though this suggests that plugins and ipc messages are now less common. Instead we see more latency probes.